# Parsing the scheduler data

In [16]:
import pandas as pd
import json
import os

In [21]:
schedule_path = 'data_eval_multicam_allocationdata_cityscapes_histacc.json'
hyps_path = 'hyp_map.json'
TASK_ID = '1'

In [27]:
# Format is:
# {
#    "sched_key":{
#       "task_id":{
#          "time":{
#             "job_id":"float"
#          }
#       }
#    }
# }
with open(schedule_path) as f:
    data = json.load(f)
with open(hyps_path) as f:
    hyperparams = json.load(f)

In [24]:
data.keys()

dict_keys(['100_1_fair_dumb_True', '100_1_thief_True', '100_1_inference_only_True', '100_2_fair_dumb_True', '100_2_thief_True', '100_2_inference_only_True', '100_4_fair_dumb_True', '100_4_thief_True', '100_4_inference_only_True', '100_8_fair_dumb_True', '100_8_thief_True', '100_8_inference_only_True', '200_1_fair_dumb_True', '200_1_thief_True', '200_1_inference_only_True', '200_2_fair_dumb_True', '200_2_thief_True', '200_2_inference_only_True', '200_4_fair_dumb_True', '200_4_thief_True', '200_4_inference_only_True', '200_8_fair_dumb_True', '200_8_thief_True', '200_8_inference_only_True'])

In [25]:
res_count = 1
scheduler="thief"
oracle="True"
period=200
schedule_key = "{}_{}_{}_{}".format(period,res_count,scheduler,oracle)

In [26]:

data[schedule_key][TASK_ID]['0']

{'jena_0_train_3_8': 0.3333333333333333,
 'zurich_1_train_6_17': 0.16666666666666666,
 'cologne_2_train_2_8': 0.0,
 'jena_0_inference': 0.16666666666666666,
 'zurich_1_inference': 0.16666666666666666,
 'cologne_2_inference': 0.16666666666666666}

In [30]:
tasks = data[schedule_key].keys()
inference_resource_weights = {}
training_resource_weights = {}
hyperparameters = {}
for task_id in tasks:
    for job_id in data[schedule_key][task_id]['0']:
        cityname = job_id.split('_')[0]
        camera_id = cityname
        resource_alloc = data[schedule_key][task_id]['0'][job_id]
        job_type = "train" if "train" in job_id else "inference"
        if job_type == "train":
            epochs = job_id.split('_')[-1]
            hp_id = job_id.split('_')[-2]
            this_hyps = hyperparams[hp_id]
            this_hyps['epochs'] = epochs
            training_resource_weights[camera_id] = resource_alloc
            hyperparameters[camera_id] = this_hyps
        elif job_type == "inference":
            inference_resource_weights[camera_id] = resource_alloc
        else:
            raise NotImplementedError

In [33]:
hyperparameters

{'jena': {'num_hidden': 64,
  'last_layer_only': True,
  'learning_rate': 0.001,
  'model_name': 'resnet18',
  'batch_size': 64,
  'subsample': 0.1,
  'momentum': 0.9,
  'epochs': '8'},
 'zurich': {'num_hidden': 64,
  'last_layer_only': True,
  'learning_rate': 0.001,
  'model_name': 'resnet18',
  'batch_size': 64,
  'subsample': 1,
  'momentum': 0.9,
  'epochs': '8'},
 'cologne': {'num_hidden': 64,
  'last_layer_only': True,
  'learning_rate': 0.001,
  'model_name': 'resnet18',
  'batch_size': 64,
  'subsample': 0.1,
  'momentum': 0.9,
  'epochs': '8'}}

In [ ]:

inference_resource_weights = {c.id: resources/len(cameras) for c in cameras}
training_resource_weights = {c.id: resources/len(cameras) for c in cameras}
hyperparameters = {c.id: self.default_hyperparams for c in cameras}